# SQL III - View
## 1. Introduction
### What is a View?
A view is a reusable virtual table that is created by querying existing tables. The virtual table is not saved in memory. Instead, each time the view is used, the query associated to the view is newly executed. 

---

### Why Are Views Important?
- **Privacy**: views can limit the information shared by taking a subset of information from full tables
- **Reusability**: can make repetitive queries easier to write
- **Generalization**: can combine different sub categories into a larger category

---

### Updatable Views 
For a view to be updatable, there must be a one-to-one relationship between the rows in the view and the rows in the underlying data. Therefore a view is updatable if it satisfies the following conditions. 

- Contains only one base relation
- Contains the key of that relation
- Does not involve any aggregate (e.g. SUM, MIN, COUNT), group by, or duplicate-elimination (e.g. DISTINCT) operations

___

### Normal Views vs Materialized Views
While normal views do not store the result of a query, materialized views store the results of a query into memory. This has the benefit of increasing the speed of data retrieval but at the cost of memory space and overhead from maintaining data consistency. The following table shows some key differences. 

| Attribute | Normal View | Materialized View | 
| --- | --- | --- |
| Data Storage | Query expression is saved with the view name. Result is always a snapshot of the state of the database at call time. | Stores the results of the query into physical storage for faster retrieval. | 
| Performance | Much slower for complex queries | Faster since results are precomputed and stored | 
| Update Behavior | Reflects the current state of the underlying data. In some cases, possible to update the underlying data from the view. | Can manually or automatically update the stored data of the view when the underlying data changes. Also possible to have a read-only or updateable view that updates the underlying data. | 
| Use Cases | Best when data is accessed infrequently and requires the most current values. | Best when data is being accessed frequently and performance is critical (e.g. reporting and analytics). | 



---

### Setup

In [74]:
# Imports
from tabulate import tabulate
import psycopg2
import time

# Database connection parameters
dbname = 'banking_db'
user = 'postgres'
password = 'postgres'
host = 'postgres_db'  # This should be the service name defined in docker-compose.yml
port = '5432'  

# Establish connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

# Create a cursor object to interact with the database
cur = conn.cursor()

### Reset Views
Run the cell below if you want to delete all views. 

In [ ]:
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        DROP VIEW public_agents
        """)
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        DROP VIEW customers_ny_toronto
        """)
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        DROP VIEW all_employees
        """)
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

## 2. Using Views
#### We will be working with 3 tables, **sales_agent**, **customers**, and **marketers**.

<img src="view_schema.png" width="800"/>

### 2.1 Privacy 
#### I want to give access to all employees in my company the name, working area, and phone number of all sales agents. 
#### I don't want to share their employee ID and commission. We can achieve this by creating a view. 

In [58]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW public_agents AS 
        SELECT employee_name, working_area, phone_no
        FROM sales_agent
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

#### Employees can now be given access to the view **public_agents** without sharing sensitive information. 
---

### 2.2 Reusability 
#### I regularly have to make queries on the customers living in the New York and Toronto. To do this I could use nested queries, where I copy the same subquery every time.

In [ ]:
# SELECT * 
# FROM 
#     (SELECT *
#     FROM customers 
#     WHERE cust_city = 'New York' OR cust_city = 'Toronto') 
# WHERE ...

#### Views make this much easier to write since we can just call the name of the view instead of copy pasting the subquery.

In [9]:
# SELECT * 
# FROM my_view
# WHERE ...

#### We start by creating the view for the query above. 

In [49]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW customers_NY_toronto AS 
        SELECT *
        FROM (SELECT c.*, s.employee_name, s.commission
              FROM customers c
              JOIN sales_agent s
              ON c.employee_id = s.employee_id)
        WHERE cust_city = 'New York' OR cust_city = 'Toronto'
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Query to fetch records
cur.execute("SELECT * FROM customers_NY_toronto")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

#### Now that I have created my view, I can query it just like any table. For example, let's say that I want to find customers who have paid more than 5000.

In [ ]:
# Query to fetch records
cur.execute("""
    SELECT * 
    FROM customers_NY_toronto
    WHERE payment_amt > 5000
""")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

### 2.3 Updatability 
#### Now we want to update payment amount of customer C00006 to 12000 by using the customers_ny_toronto view. 

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        UPDATE customers_NY_toronto
        SET payment_amt = 12000
        WHERE cust_id = 'C00006';
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        DELETE FROM customers_NY_toronto
        WHERE cust_id = 'C00006';
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        INSERT INTO customers_NY_toronto
        VALUES ('C00007', 'Hamish', 'New York', '4000', 'A004', 'Ivan', '0.15')
        """)
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

#### We can not update, delete, or insert a new entry into the **customers_ny_toronto** view because it is based on more than one table.
---
#### Now we want to change the working city of the sales agent with the name John from our **public_agents** view.

In [ ]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

In [64]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        UPDATE public_agents
        SET working_area = 'Zürich'
        WHERE employee_name = 'John';
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

#### We can try updating the underlying table to see how the view changes. Let's delete 'John' from the **sales_agent** table. 

In [72]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        DELETE FROM sales_agent CASCADE
        WHERE employee_id = 'A001';
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM public_agents")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

#### Updating the underlying **sales_agent** table automatically affects the **public_agents** view.
---

### 2.4 Generalization
#### Generalization can help create views that combine two similar categories into one. In this example, we want to query a table containing both agents and contractors. To do this, we can combine the two tables into one view called **all_employees**. 

In [36]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try: 
    # Create view
    cur.execute("""
        CREATE VIEW all_employees AS 
        SELECT employee_id, employee_name, working_area, phone_no 
        FROM sales_agent
        UNION
        SELECT employee_id, employee_name, working_area, phone_no
        FROM marketers
        """)
    
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    
except Exception as e:
    # In case of error, rollback transaction
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")

In [ ]:
# Query to fetch records 
cur.execute("SELECT * FROM all_employees")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

---
### Run cell below to reset to initial setup

In [ ]:
# Delete all tables
cur.execute("BEGIN TRANSACTION;")

cur.execute("""
    DROP TABLE IF EXISTS sales_agent CASCADE;
    """)
cur.execute("""
    DROP TABLE IF EXISTS customers CASCADE;
    """)
cur.execute("""
    DROP TABLE IF EXISTS marketers CASCADE;
    """)

cur.execute("COMMIT;")

# Create tables
cur.execute("BEGIN TRANSACTION;")

cur.execute("""
    CREATE TABLE sales_agent(	
        EMPLOYEE_ID char(6) NOT NULL PRIMARY KEY, 
        EMPLOYEE_NAME char(40), 
        WORKING_AREA char(35), 
        COMMISSION numeric(10,2), 
        PHONE_NO char(15)
    );
    """)

cur.execute("""
    INSERT INTO sales_agent 
    VALUES 
        ('A001', 'John', 'Bangalore', '0.14', '077-12346674'),
        ('A002', 'Mukesh', 'Mumbai', '0.11', '029-12358964'),
        ('A003', 'Alex ', 'London', '0.13', '075-12458969'),
        ('A004', 'Ivan', 'Toronto', '0.15', '008-22544166'),
        ('A005', 'Anderson', 'Brisbane', '0.13', '045-21447739'),
        ('A006', 'McDen', 'London', '0.15', '078-22255588'),
        ('A007', 'Ramasundar', 'Bangalore', '0.15', '077-25814763'),
        ('A008', 'Alford', 'New York', '0.12', '044-25874365'),
        ('A009', 'Benjamin', 'Hampshair', '0.11', '008-22536178'),
        ('A010', 'Santakumar', 'Chennai', '0.14', '007-22388644'),
        ('A011', 'Ravi Kumar', 'Bangalore', '0.15', '077-45625874'),
        ('A012', 'Lucida', 'San Jose', '0.12', '044-52981425');
    """)

cur.execute("""
    CREATE TABLE customers(	
        CUST_ID varchar(6) NOT NULL PRIMARY KEY, 
        CUST_NAME varchar(40) NOT NULL, 
        CUST_CITY char(35), 
        PAYMENT_AMT numeric(12,2) NOT NULL, 
        EMPLOYEE_ID char(6) NOT NULL,
        FOREIGN KEY (EMPLOYEE_ID)
            REFERENCES sales_agent
            ON DELETE CASCADE
    ); 
    """)

cur.execute("""
    INSERT INTO customers 
    VALUES 
        ('C00013', 'Holmes', 'London', '6000.00', 'A003'),
        ('C00001', 'Micheal', 'New York', '3000.00', 'A008'),
        ('C00020', 'Albert', 'New York', '5000.00', 'A008'),
        ('C00025', 'Ravindran', 'Bangalore', '5000.00', 'A011'),
        ('C00024', 'Cook', 'London', '4000.00', 'A006'),
        ('C00015', 'Stuart', 'London', '6000.00', 'A003'),
        ('C00002', 'Bolt', 'New York', '5000.00', 'A008'),
        ('C00018', 'Fleming', 'Brisbane', '7000.00', 'A005'),
        ('C00021', 'Jacks', 'Brisbane', '7000.00', 'A005'),
        ('C00019', 'Yearannaidu', 'Bangalore', '8000.00', 'A010'),
        ('C00005', 'Sasikant', 'Dehli','7000.00', 'A002'),
        ('C00007', 'Ramanathan', 'Dehli','7000.00', 'A010'),
        ('C00022', 'Avinash', 'Mumbai', '7000.00', 'A002'),
        ('C00004', 'Winston', 'Brisbane', '5000.00', 'A005'),
        ('C00023', 'Karl', 'London', '4000.00', 'A006'),
        ('C00006', 'Shilton', 'Toronto', '10000.00', 'A004'),
        ('C00010', 'Charles', 'Hampshair', '6000.00', 'A009'),
        ('C00017', 'Srinivas', 'Bangalore', '8000.00', 'A007'),
        ('C00012', 'Steven', 'San Jose', '5000.00', 'A012'),
        ('C00008', 'Karolina', 'Toronto', '7000.00', 'A004'),
        ('C00003', 'Martin', 'Toronto', '8000.00', 'A004'),
        ('C00009', 'Ramesh', 'Mumbai', '8000.00', 'A002'),
        ('C00014', 'Rangarappa', 'Bangalore', '8000.00', 'A001'),
        ('C00016', 'Venkatpati', 'Bangalore', '8000.00', 'A007'),
        ('C00011', 'Sundariya', 'Chennai', '7000.00', 'A010');
    """)

cur.execute("""
    CREATE TABLE marketers(	
        EMPLOYEE_ID char(6) NOT NULL PRIMARY KEY, 
        EMPLOYEE_NAME char(40), 
        WORKING_AREA char(35), 
        SALARY numeric(10,2), 
        PHONE_NO char(15)
    );
    """)

cur.execute("""
    INSERT INTO marketers 
    VALUES 
        ('A013', 'John', 'New York', '50000', '077-11124769'),
        ('A014', 'Sarah ', 'San Francisco', '52000', '032-89790125'),
        ('A015', 'Liam', 'Dublin', '38000', '011-33447076'),
        ('A016', 'Richard', 'Vancouver', '47000', '008-44720826'),
        ('A017', 'Whitaker', 'New York', '54000', '012-58859201'),
        ('A018', 'Lucia', 'San Jose', '49000', '032-81027792');
    """)

cur.execute("COMMIT;")

In [ ]:
# Query to fetch records 
cur.execute("SELECT * FROM sales_agent")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))